# Classical Mechanics I (PHYS 311)
## Studio 8

*Name:* 

*Date:* 

## Introduction to animations

Over the next few studios, we're going to work on visualizing central force problems and orbits. To do that, we'll start off by learning how to do some simple animations in `matplotlib`.

In [ ]:
%pylab inline

We're going to need some useful packages provided by matplotlib and to link a standard Linux/UNIX video-editing tool `ffmpeg` which is luckily installed on ISAAC for us.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display

Here's a simple example of how to animate something. We need to define a function that given a frame number (i.e. a single point in time), gives back an object we can draw.

We then hand that to this fancy function `FuncAnimation()` who will take care of the hard part. Then we just need to convert its output to something we can display in the notebook.

For ease of use later, let's also wrap this in a function that can be called concisely.

In [ ]:
def plotSomeStuff(x_array, y_array, nframes=100, ax = None, fig = None):
    if ax == None or fig == None:
        fig, ax = plt.subplots()
        ax.set_xlim(-10,10)
        ax.set_ylim(-10, 10)

    phi = linspace(0,2*np.pi,100)

    marker1, = ax.plot([],"o") # Let's create an empty plot that has a circle marker
    orbit1, = ax.plot([]) # Let's create an empty plot to eventually become an orbit line

    def animate(frame_num):
        """
        """
        x = x_array[frame_num]
        y = y_array[frame_num]
        
        # Let's put a marker at the last position of these arrays.
        # We're using set_data which is updating the data stored in the marker1 object.
        marker1.set_data([x,y])
        return

    anim = FuncAnimation(fig, animate, frames=nframes, interval=20)
    display(HTML(anim.to_jshtml()))
    plt.close()

nframes = 100
tmax = 10
t_array = linspace(0,tmax,nframes)
r = 2
x_array = r*cos(t_array/10 * 2 * np.pi)
y_array = r*sin(t_array/10 * 2 * np.pi)

# Now we can run the function
plotSomeStuff(x_array,y_array,nframes)

Now copy the code above to below, and try to get an animation that shows the marker moving in a circle, but also where it's been at earlier times. I've already created an object `orbit1` for you to fill up. Inside `animate`, the `x` and `y` arrays already store this information, so just use `set_data` on the `orbit1` object to get it to trace out the path of this thing.

For this, you'll need to recall how access the first N elements of a numpy array.

Now we can use your new `plotSomeStuff()` function to plot some more things.

## Using animations for visualizing physics

Now that we have that tool, let's use this for some physics. Recall that we have Newton's 2nd law in polar coordinates:

$$m(\ddot r - r \dot \phi^2) = \sum F_r$$
$$m(r\ddot \phi + 2 \dot r \dot \phi) = \sum F_\phi = 0$$

where $F_\phi$ is zero for central forces. This formulation was useful for calculating by hand since it gets into a basis where the Force is "1-dimensional". However, the computer doesn't care! It'll solve it in whatever basis we want. So we can go back and define our 2D problem in cartesian coordinates.

$$m\ddot x = \sum F_x $$
$$m\ddot y = \sum F_y $$

We know how to solve systems of ODEs from past studios, so let's code up an `odeint` implementation of this. 

Let's start off with $F_r=-r$ to model the motion of a 2-dimensional harmonic oscillator.

In [ ]:
from scipy.integrate import odeint

In [ ]:

def ode_system(inputs,t,m):
    """
    This function represents a series of first order ODEs.
    
    Return: List of expressions for the first time derivative of the inputs, in order.
    """
    
    # Parse the inputs list to positions x,y and vector magnitudes xdot,ydot
    [x,y,xdot,ydot] = inputs
    
    r = np.hypot(x,y) # This is a handy function that computes sqrt(x^2+y^2) for you
    phi = np.arctan2(y,x) # And this calculates the polar angle of our vector

    # 2D Hooke's law as a central force
    Fr = -r
    Fx = Fr*np.cos(phi)
    Fy = Fr*np.sin(phi)
    
    # Return the expressions for the time-derivatives of the inputs.
    return [xdot,ydot,Fx/m,Fy/m]


nframes = 100
tmax = 10
t_array = linspace(0,tmax,nframes)

# Solve the ODEs with an initial condition of:
# x0 = 2, y0 = 2, xdot0 = 0, ydot0 = 0
solutions = odeint(ode_system, (2,2,0,0), t_array, args=(0.1,))

x_array = solutions[:,0]
y_array = solutions[:,1]

# Now we can run the function
plotSomeStuff(x_array,y_array,nframes)

Now solve the same system, but releasing the particle from some point in this space, but now with a nonzero initial velocity.

Now let's switch from a harmonic oscillator force to that of gravity. Create a new `ode_system_grav()` and instead of `Fr=-r`, make the force `Fr=-1/(r*r)`. Then we'll be playing with an inverse square force. Do this and draw an example trajectory, maybe with initial velocity of zero.

Play with the parameters to find a closed orbit. Some sort of ellipse. Notice the time evolution of the orbit. See how there are portions of the orbit where the particle moves faster? Try an additional set of initial conditions and come up with another closed orbit. 

Now increase the kinetic energy (i.e. increase the initial total speed) and watch the orbit become unbounded. Show a situation where the particle gets slingshotted off to infinity.

**EC(+1): For a bounded orbit, draw the area of the ellipse swept out in some time $\Delta t$. Show how this area is a constant (because angular momentum is conserved) and therefore demonstrate Kepler's second law. (Basically draw the blue triangle shown in the animation at https://en.wikipedia.org/wiki/Kepler%27s_laws_of_planetary_motion#Second_law)**